# PART 2: Design a python based interactive semi - rule based chatbot.

In [ ]:
# Import Libraries 
import nltk
import random
import numpy as np
import json
from nltk.tokenize import word_tokenize 
nltk.download('omw-1.4')

In [ ]:
nltk.download('punkt') # to tokenize sentance
nltk.download('wordnet') # Download to Lammetaize

In [12]:
# Load json corpus file in python
corpus1=open('GL Bot.json','r') 
corpus=json.load(corpus1)
corpus1.close()

In [ ]:
corpus

In [14]:
# code creats word list and all possible class list
words=[] # Contains all unique words
classes=[] #Contains all unique classes
document=[] # Contains all questions and tag i.e. class with which it belongs.
for intent in corpus['intents']: # Complete intent values out of key value is stored in intent
  for pattern in intent['patterns']:# All sentances within patterns are collected at one place
    w=word_tokenize(pattern) # performs word tokenization on each pattern
    words.extend(w)# used to append all individual words in the list
    document.append((w,intent['tag'])) #creat a set containing tokenised words of a pattern with its respective tag
    #print(document)
    if intent['tag'] not in classes:
      classes.append(intent['tag'])# creating list of all possible labels

In [ ]:
classes

In [ ]:
document

In [ ]:
words

In [19]:
# Performs word lemmatization to form a list of unique words and targets
from nltk.stem import WordNetLemmatizer
ignore=['?']
lemmatizer=WordNetLemmatizer()
words=[lemmatizer.lemmatize(w.lower())for w in words if w not in ignore] # From word list take each word except ?, lower it and convert it into its root word.
# removes duplicate words from list and prepares vocabulay containing unique words in alphabatical order
word=sorted(list(set(words)))
# sort classes alphabatically
classes=sorted(classes)

In [ ]:
# Performs vectorization of tokens using Bag of word technique
training=[] # Contains BOW related to each question in pattern
one_hot_result=[] #Contains one hot vector of class
empty=[0]*len(classes)
# Performs Bag of word operation on every row of document
for doc in document:
  bag=[] #empty list of each sentance
  doc1=doc[0] # take the question pattern. 
  new_pattern=[lemmatizer.lemmatize(w.lower()) for w in doc1] # convert each word of a question into its root word
  #print(new_pattern)
  for x in word:
    bag.append(1) if x in new_pattern else bag.append(0) # if bag containts word label it 1 otherwise 0.
# Performs One hot encoding of Label    
  one_hot_result=list(empty)
  one_hot_result[classes.index(doc[1])]=1
  #print(one_hot_result)
  training.append([bag,one_hot_result])
  print(training)
training=np.array(training)
# Seprate Features and Labels
x_train=list(training[:,0])
y_train=list(training[:,1])

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [21]:
# DNN model for classification
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout

model=Sequential()
model.add(Dense(64,input_dim=len(x_train[0]),activation='relu')) #  i nput dim is no of unique words
model.add(Dropout(0.5))
model.add(Dense(32,input_dim=64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8,input_dim=32,activation='softmax')) # o/p dims are 8 here

In [22]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
# Training DLL model and Learning parameters for future use
model.fit(x_train,y_train,epochs=500,batch_size = 16)
model.save("model.tflearn")

Epoch 1/500
8/8 [==============================] - 3s 15ms/step - loss: 2.1057 - accuracy: 0.1406
Epoch 2/500
8/8 [==============================] - 0s 3ms/step - loss: 2.0760 - accuracy: 0.1797
Epoch 3/500
8/8 [==============================] - 0s 3ms/step - loss: 2.0297 - accuracy: 0.1953
Epoch 4/500
8/8 [==============================] - 0s 3ms/step - loss: 2.0388 - accuracy: 0.1562
Epoch 5/500
8/8 [==============================] - 0s 3ms/step - loss: 2.0398 - accuracy: 0.1719
Epoch 6/500
8/8 [==============================] - 0s 5ms/step - loss: 1.9910 - accuracy: 0.2266
Epoch 7/500
8/8 [==============================] - 0s 3ms/step - loss: 1.9599 - accuracy: 0.2266
Epoch 8/500
8/8 [==============================] - 0s 3ms/step - loss: 1.9694 - accuracy: 0.2344
Epoch 9/500
8/8 [==============================] - 0s 3ms/step - loss: 1.9797 - accuracy: 0.2500
Epoch 10/500
8/8 [==============================] - 0s 2ms/step - loss: 1.9047 - accuracy: 0.3047
Epoch 11/500
8/8 [==========

INFO:tensorflow:Assets written to: model.tflearn\assets


INFO:tensorflow:Assets written to: model.tflearn\assets


In [27]:
# performs tokenization on user input
#a.tokenizing and lemmatizing
def sentance_processor(sentance):
  sent=word_tokenize(sentance)
  user_sent=[lemmatizer.lemmatize(w.lower())for w in sent]
  return user_sent

In [28]:
# Performs BOW on user input
def bow(sentance,word):
  sentance=sentance_processor(sentance)
  bag=[0]*len(word)
  for x in sentance:
    #print('sen:',x)
    for i,w in enumerate(word):
      if w==x:
        #print(w)
        bag[i]=1
  return (np.array(bag))

In [29]:
# Function predicts class for user input using DLL paramerets
import tensorflow as tf
ERROR_THRESHOLD = 0.30 # O/P below 30 match are not considered  
def classify(sentence):
  dim=tf.expand_dims([bow(sentence, word)], axis=1)[0]
    # generate probabilities from the model
  results = model.predict(dim)
    #print('result', result)
    # filter out predictions below a threshold
  results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
    print(return_list)
    # return tuple of intent and probability
  return return_list

def response(sentence, show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in corpus['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                  return print("ROBO:",random.choice(i['responses']))

In [30]:
def chat():
  flag=True
  print("ROBO:I am Sandip.")
  print( "(To exit: Type quit)")
  while(flag==True):
    print("\n")
    user_response = input()
    user_response=user_response.lower()
    if (user_response != 'quit'):
      response(user_response)
    else:
      flag=False
      return print("ROBO: Bye! take care..")

In [ ]:
chat()

In [ ]:
classify('how are you?')

In [ ]:
response('how are you?')